In [1]:
%cd ..
%cd Dataset/audio
%pwd

/home/deepankaracharyya/TTS-exp
/home/deepankaracharyya/TTS-exp/Dataset/audio


'/home/deepankaracharyya/TTS-exp/Dataset/audio'

In [9]:
log_dir = "Iteration-03-Trixie-12June"

In [5]:
import os

def get_best_ckpt_from_last_run(base_dir):    
    models = {
            'FastPitch' : -1,
            'HifiGan':-1
    }
    
    results= []
    for model_name in ['FastPitch','HifiGan']:
        exp_dirs = os.path.join(os.getcwd(),base_dir,model_name)
        for (dir_path, dir_names, file_names) in os.walk(exp_dirs):
            for f in file_names:
                if 'last.ckpt' in str(f):
                    models[model_name] = os.path.join(dir_path,f)
                    
    return models

In [10]:
# TODO(oktai15): remove +model.text_tokenizer.add_blank_at=true when we update FastPitch checkpoint
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=./fastpitch_train.json \
  validation_datasets=./fastpitch_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/whitelist.tsv \
  exp_manager.exp_dir=./$log_dir \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=2000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=5 \
  model.train_ds.dataloader_params.batch_size=16 model.validation_ds.dataloader_params.batch_size=16 \
  model.n_speakers=1 model.pitch_mean=110 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-06-12 17:26:01 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-12 17:26:04 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-06-12 17:26:07 exp_manager:287] Experiments will be logged at Iteration-02-Trixie-12June/FastPitch/2022-06-12_17-26-07
[NeMo I 2022-06-12 17:26:07 exp_manager:661] TensorboardLogger has been set up
[NeMo W 2022-06-12 17:26:07 nemo_logging:349] /usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:2319: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecat

In [11]:
#imports
import os
import json
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
import soundfile as sf
import numpy as np

[NeMo W 2022-06-12 17:58:59 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-12 17:59:00 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [15]:
chck_pts = get_best_ckpt_from_last_run(log_dir)

FP_chck_pt = chck_pts['FastPitch']
spec_model = FastPitchModel.load_from_checkpoint(FP_chck_pt)
spec_model.eval().cuda()

[NeMo I 2022-06-12 18:00:06 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-06-12 18:00:57 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-06-12 18:00:57 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-06-12 18:00:59 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: ./fastpitch_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
 

[NeMo I 2022-06-12 18:00:59 features:200] PADDING: 1


FastPitchModel(
  (mel_loss): MelLoss()
  (pitch_loss): PitchLoss()
  (duration_loss): DurationLoss()
  (aligner): AlignmentEncoder(
    (softmax): Softmax(dim=3)
    (log_softmax): LogSoftmax(dim=3)
    (key_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(384, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(768, 80, kernel_size=(1,), stride=(1,))
      )
    )
    (query_proj): Sequential(
      (0): ConvNorm(
        (conv): Conv1d(80, 160, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (1): ReLU()
      (2): ConvNorm(
        (conv): Conv1d(160, 80, kernel_size=(1,), stride=(1,))
      )
      (3): ReLU()
      (4): ConvNorm(
        (conv): Conv1d(80, 80, kernel_size=(1,), stride=(1,))
      )
    )
  )
  (forward_sum_loss): ForwardSumLoss(
    (log_softmax): LogSoftmax(dim=3)
    (ctc_loss): CTCLoss()
  )
  (bin_loss): BinLoss()
  (preprocessor): AudioToMelSpectrogramPreprocessor(
  

In [17]:
!rm -r ./train_mels ./dev_mels

rm: cannot remove './train_mels': No such file or directory
rm: cannot remove './dev_mels': No such file or directory


In [20]:
#helper functions
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

In [21]:
#dataset preparation
# Get records from the training manifest
manifest_path = "./fastpitch_train.json"
records = []
with open(manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path("./train_mels")
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
        
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = "hifigan_train_ft.json"
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [22]:
# Get records from the training manifest
manifest_path_validation = "./fastpitch_val.json"
records_val = []
with open(manifest_path_validation, "r") as f:
    for i, line in enumerate(f):
        records_val.append(json.loads(line))

save_dir_val = Path("./dev_mels")
save_dir_val.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records_val):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
       
    with torch.no_grad():
        if "normalized_text" in r:
            text = spec_model.parse(r["normalized_text"], normalize=False)
        else:
            text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
        )[0]
        
        save_path_val = save_dir_val / f"mel_{i}.npy"
        np.save(save_path_val, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path_val)

hifigan_val_manifest_path = "hifigan_val_ft.json"
with open(hifigan_val_manifest_path, "w") as f:
    for r in records_val:
        f.write(json.dumps(r) + '\n')

## Restart the kernel

In [1]:
%cd ..
%cd Dataset/audio
%pwd

/home/deepankaracharyya/TTS-exp
/home/deepankaracharyya/TTS-exp/Dataset/audio


'/home/deepankaracharyya/TTS-exp/Dataset/audio'

In [3]:
log_dir = "Iteration-02-Trixie-12June"

In [4]:
#finetuning the hifigan model
!(python hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=16 \
model.max_steps=1000 \
model.optim.lr=0.0002 \
~model.optim.sched \
train_dataset=./hifigan_train_ft.json \
validation_datasets=./hifigan_val_ft.json \
exp_manager.exp_dir=$log_dir \
+init_from_nemo_model=tts_hifigan.nemo \
trainer.check_val_every_n_epoch=2 \
)

[NeMo W 2022-06-12 18:34:02 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-12 18:34:03 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-06-12 18:34:04 exp_manager:287] Experiments will be logged at Iteration-02-Trixie-12June/HifiGan/2022-06-12_18-34-04
[NeMo I 2022-06-12 18:34:04 exp_manager:661] TensorboardLogger has been set up
[NeMo W 2022-06-12 18:34:04 nemo_logging:349] /usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:2319: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated 

# Checking out the newly trained model :

In [5]:
#imports
import os
import json
import librosa

import torch
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
from pathlib import Path

from nemo.collections.tts.models import FastPitchModel
from nemo.collections.tts.models import HifiGanModel

import soundfile as sf
import numpy as np


[NeMo W 2022-06-12 18:53:32 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-06-12 18:53:33 experimental:28] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [6]:
def get_best_ckpt_from_last_run(base_dir):    
    models = {
            'FastPitch' : -1,
            'HifiGan':-1
    }
    
    results= []
    for model_name in ['FastPitch','HifiGan']:
        exp_dirs = os.path.join(os.getcwd(),base_dir,model_name)
        for (dir_path, dir_names, file_names) in os.walk(exp_dirs):
            for f in file_names:
                if 'last.ckpt' in str(f):
                    models[model_name] = os.path.join(dir_path,f)
                    
    return models

In [7]:
chck_pts = get_best_ckpt_from_last_run(log_dir)
FP_chck_pt = chck_pts['FastPitch']
HFG_chck_pt = chck_pts['HifiGan']

print(FP_chck_pt)
spec_model = FastPitchModel.load_from_checkpoint(FP_chck_pt)
spec_model.eval().cuda()

print(HFG_chck_pt)
vocoder = HifiGanModel.load_from_checkpoint(HFG_chck_pt)
vocoder = vocoder.eval().cuda()

/home/deepankaracharyya/TTS-exp/Dataset/audio/Iteration-02-Trixie-12June/FastPitch/2022-06-12_17-26-07/checkpoints/FastPitch--v_loss=2.2250-epoch=29-last.ckpt
[NeMo I 2022-06-12 18:53:40 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-06-12 18:54:30 experimental:28] Module <class 'nemo.collections.tts.torch.g2ps.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-06-12 18:54:31 g2ps:87] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-06-12 18:54:33 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: ./fastpitch_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
 

[NeMo I 2022-06-12 18:54:33 features:200] PADDING: 1
/home/deepankaracharyya/TTS-exp/Dataset/audio/Iteration-02-Trixie-12June/HifiGan/2022-06-12_18-34-04/checkpoints/HifiGan--val_loss=0.4684-epoch=7-last.ckpt


[NeMo W 2022-06-12 18:54:37 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: ./hifigan_train_ft.json
      sample_rate: 22050
      n_segments: 8192
      max_duration: null
      min_duration: 0.75
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 16
      num_workers: 4
    
[NeMo W 2022-06-12 18:54:37 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: ./hifigan_val_ft.json
      sample_rate: 22050
      n_s

[NeMo I 2022-06-12 18:54:37 features:200] PADDING: 0
[NeMo I 2022-06-12 18:54:37 features:208] STFT using exact pad
[NeMo I 2022-06-12 18:54:37 features:200] PADDING: 0
[NeMo I 2022-06-12 18:54:37 features:208] STFT using exact pad


In [8]:
#helper functions
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def custom_infer(transcript, spec_model, vocoder):
  spec, audio = infer(spec_model, vocoder, transcript)
  print("\n\n\n\n")
  ipd.display(ipd.Audio(audio, rate=22050))
  %matplotlib inline
  plt.show()

In [9]:
custom_infer("Its a wonderful day to train some models", spec_model, vocoder)